![Alt Text](axpo-animated.gif)
# Witamy na rozmowie rekrutacyjnej
## na stanowisko: **Financial Data Analyst** w **Axpo Polska**

Cieszymy się, że zdecydowałeś/aś się wziąć udział w naszym procesie rekrutacyjnym. Przedstawiamy Ci case study, w którym zaprezentujesz swoje umiejętności analityczne.

---

## Opis Case Study

W bazie (tworzonej automatycznie przy uruchomieniu) znajdują się trzy tabele, przeznaczone do porównania danych między systemami:

- **hd_readings** – miesięczne odczyty zużyć dla ok. 500 POD-ów (medium_id: 1=power, 2=gas), zakres 2022-01 .. 2025-09.
  - Dokładnie 100 wierszy ma `loading_date = wczoraj`, pozostałe mają `loading_date = dziś`.
  - `real_usage_mwh = forecast_usage_mwh * U(0.6 .. 1.3)`.
  - `product_id` spójny w ramach POD + kontraktu; kontrakty maks. 24 mies.; przy przejściach kontraktów część POD-ów zmienia produkt.
  - Ciągłość miesięcy: `usage_from_date` = 1. dzień miesiąca, `usage_to_date` = ostatni dzień miesiąca.
- **product_dict** – słownik ~100 produktów, nazwy zawierają kluczowe frazy: `gas_fix`, `gas_spot`, `power_fix`, `power_spot`.
- **endur_dump** – agregat portfela na dzisiejszy dzień: (BOOK, endur_product, delivery_month, volume).
  - Powstaje na podstawie dzisiejszych wierszy z HD, więc po poprawnym mapowaniu i filtrach wolumeny powinny się zgadzać.

---

## Zadania do Wykonania
- Uruchom komórkę poniżej – wygeneruje unikatowe dane i załaduje je do silnika SQL w pamięci (SQLite) oraz pozostawi jako DataFrame’y Pythona.
- Zmapuj `product_name` -> `endur_product` (GAS_FIX / GAS_SPOT / POWER_FIX / POWER_SPOT) i `BOOK` (AXPL_SME_GAS dla GAS_*, AXPL_SME_POWER dla POWER_*).
- Przefiltruj HD do `loading_date = dziś` i zagreguj do `(BOOK, endur_product, delivery_month)`, sumując `real_usage_mwh`.
- Porównaj wynik z `endur_dump` (`reporting_date = dziś`) – wolumeny powinny się zgadzać.
- Zadanie z gwiazdką: wylistuj POD-y, które zmieniały produkt w historii (per medium), opcjonalnie wskaż miesiące zmiany.

## Dostępne narzędzia
- Analizę możesz przeprowadzić używając Pythona, SQL-a lub obu narzędzi.
- Możesz korzystać z internetu, ale nie używaj generatywnej AI.
- Bardziej niż wynik oceniany będzie sposób jego osiągnięcia oraz tok myślenia.

## Podpowiedź
- Po właściwym mapowaniu i filtrze do dzisiejszego `loading_date`, różnica HD vs Endur powinna wynosić 0 na każdym poziomie agregacji.

---
# Powodzenia!


In [5]:
import sqlite3
import pandas as pd
from IPython.display import display

# Spróbuj standardowego importu; jeśli się nie uda, załaduj plik lokalnie
try:
    from db_setup import setup_database, modify_random_usage
except ImportError as e:
    import importlib.util, os
    module_path = os.path.join(os.getcwd(), 'db_setup.py')
    if not os.path.exists(module_path):
        raise ImportError(f'Nie znaleziono db_setup.py w {os.getcwd()}') from e
    spec = importlib.util.spec_from_file_location('db_setup', module_path)
    db_setup = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(db_setup)
    setup_database = db_setup.setup_database
    modify_random_usage = getattr(db_setup, 'modify_random_usage', None)

# Wygeneruj dane (za każdym uruchomieniem inne)
hd_readings, product_dict, endur_dump = setup_database()

print("Przykładowe dane HD (hd_readings):")
display(hd_readings.head())

print("\nPrzykładowy słownik produktów (product_dict):")
display(product_dict.head())

print("\nPrzykładowy agregat Endur (endur_dump):")
display(endur_dump.head())

# (Opcjonalnie) zaburzenie danych jak w oryginalnym repo (jeśli funkcja istnieje w Twoim db_setup.py)
if modify_random_usage is not None:
    info = modify_random_usage(hd_readings)

# Rejestracja do in-memory SQLite (identycznie jak w oryginalnym repo)
conn = sqlite3.connect(':memory:')
hd_readings.to_sql('hd_readings', conn, index=False)
product_dict.to_sql('product_dict', conn, index=False)
endur_dump.to_sql('endur_dump', conn, index=False)

def sqlquery(query, conn=conn):
    return pd.read_sql_query(query, conn)


Przykładowe dane HD (hd_readings):


,loading_date,pod_header_id,customer_id,adres_id,contrct_id,product_id,medium_id,usage_from_date,usage_to_date,forecast_usage_mwh,real_usage_mwh
0,2025-10-21,POD000175,CUST000070,ADDR000264,CTR000001,PRD008,2,2022-01-01,2022-01-31,13.681,11.416
1,2025-10-21,POD000175,CUST000070,ADDR000264,CTR000001,PRD008,2,2022-02-01,2022-02-28,30.256,39.038
2,2025-10-21,POD000175,CUST000070,ADDR000264,CTR000001,PRD008,2,2022-03-01,2022-03-31,24.558,21.064
3,2025-10-21,POD000175,CUST000070,ADDR000264,CTR000001,PRD008,2,2022-04-01,2022-04-30,19.044,11.532
4,2025-10-21,POD000175,CUST000070,ADDR000264,CTR000001,PRD008,2,2022-05-01,2022-05-31,16.466,20.131



Przykładowy słownik produktów (product_dict):


,product_id,product_name
0,PRD001,ultra_001_gas_fix
1,PRD002,gas_fix_002_premium
2,PRD003,ultra_003_gas_fix
3,PRD004,gas_fix_004_premium
4,PRD005,ultra_005_gas_fix



Przykładowy agregat Endur (endur_dump):


,reporting_date,BOOK,endur_product,delivery_month,volume
0,2025-10-21,AXPL_SME_GAS,GAS_FIX,2022-01,2447.229
1,2025-10-21,AXPL_SME_GAS,GAS_FIX,2022-02,2400.023
2,2025-10-21,AXPL_SME_GAS,GAS_FIX,2022-03,2543.045
3,2025-10-21,AXPL_SME_GAS,GAS_FIX,2022-04,2488.948
4,2025-10-21,AXPL_SME_GAS,GAS_FIX,2022-05,2542.328


In [ ]:
query = """
with hd_data as (
SELECT 
    hd.* , pd.product_name,
    case when pd.product_name like '%gas_fix%' then 'gas_fix'
            when pd.product_name like '%gas_spot%' then 'gas_spot'
            when pd.product_name like '%power_fix%' then 'power_fix'
            when pd.product_name like '%power_spot%' then 'power_spot'
            else 'other' end as endur_product_type
FROM 
    hd_readings hd
    
    JOIN product_dict pd ON hd.product_id = pd.product_id
WHERE 
    1=1
    and hd.loading_date = '2025-10-21'
    ), hd_aggregated as ( 
select 
    endur_product_type,  substr(usage_from_date, 1,7) as usage_month,
    
    sum(real_usage_mwh) as real_usage_mwh
from 
    hd_data
group by 
    1,2
order by 
    endur_product_type, 2) , joined as (
    select * from hd_aggregated 
    left join endur_dump ed 
    on hd_aggregated.endur_product_type = lower(ed.endur_product) and hd_aggregated.usage_month = ed.delivery_month
) 
select * from joined 
where real_usage_mwh - volume <> 0
    
"""
sqlquery(query)

,endur_product_type,usage_month,real_usage_mwh,reporting_date,BOOK,endur_product,delivery_month,volume
0,gas_fix,2025-05,2452.768,2025-10-21,AXPL_SME_GAS,GAS_FIX,2025-05,2453.955
1,power_spot,2023-10,2756.144,2025-10-21,AXPL_SME_POWER,POWER_SPOT,2023-10,2756.144
